In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import datetime
import seaborn as sn
from shapely import wkb
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gp
import matplotlib.pyplot as plt
import hashlib
from scipy.spatial.distance import cdist
import os
%matplotlib inline
from collections import deque
import networkx as nx
from time import sleep
from copy import deepcopy
from sympy import Matrix, init_printing
import pickle

In [2]:
# matches = pd.read_csv('./data-test/mr.csv', sep=';')
# matches = matches[matches.cpath.isnull()]
# matches['tt'] = matches['ep'].apply(lambda x: np.array([float(e) for e in x.split(',')])) * \
# matches['tp'].apply(lambda x: np.array([float(e) for e in x.split(',')]))
# matches['error'] = matches['error'].apply(lambda x: [float(x) for x in x.split(',')])
# matches['match'] = matches['error'].apply(lambda x: True if all(y<0.00001 for y in x) else False)
# matches = matches[matches['match']]

In [3]:
def geo_center(S, vertices):
    S = list(S)
    mean_point = np.mean(vertices[S], axis=0)
    dists = np.linalg.norm(vertices[S]-mean_point, axis=1)
#     print(f'arg min to geocenter: {np.argmin(dists)}')
    center = S[np.argmin(dists)]
    return center

def adjacent_vertex(S, adjacencies):
    out_vertices = set()
    for p in S:
        adjacents = set(np.where(adjacencies[p]==1)[0])
        complements = adjacents-S
        out_vertices = out_vertices.union(complements)
    return out_vertices

In [4]:
trajectories = gp.read_file('./data-test/trajectories/trajs.shp')
trajectories.set_index('id', drop=True, inplace=True)

In [ ]:
np.random.seed(123456)
newarray = np.concatenate(np.array(trajectories['geometry'].apply(lambda x: np.array(x.coords))))
Rv = 0.00015
V_set = []
while newarray.size > 0:
    rand_point = newarray[np.random.choice(newarray.shape[0], size=1, replace=False)][0]
    V_set.append(rand_point)
    dists = np.linalg.norm(newarray-rand_point, axis=1)
    newarray = newarray[dists>Rv]
V_set = np.vstack(V_set)

Rc = 0.0009
dists = cdist(V_set, V_set)
adj_matrix = np.ma.getmask(np.ma.masked_where(dists<Rc, dists)).astype(np.int)
np.fill_diagonal(adj_matrix, 0)



r = 0.0010
delta = 6
indices = np.arange(0, V_set.shape[0], 1)
all_marked = set()
all_nodes = set(range(V_set.shape[0]))
random_choices = []

while all_nodes != all_marked:
#     s = np.random.choice(indices)
    s = np.random.choice(indices[~np.isin(indices, random_choices)])
    random_choices.append(s)
    Q_g = deque()
    Q_g.appendleft(s)
    Q_l = deque()
    # eddited on the original algorithm
    Q_l.appendleft(s)
    distances = deque()
    distances.appendleft(0)
    count = 0
    S = set()
    marks = np.zeros((V_set.shape[0]), np.int)


    graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices}
    with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
        pickle.dump(graph_dict, graph_file)
    sleep(2)
    
    marks[s] = 1
    S.add(s)
    print(f'set S is: {S}')

    while len(Q_g) != 0:
        print(f'Q_g currently is: {Q_g}')
        print(f'Q_l currently is: {Q_l}')
        seed = Q_g.pop()
        print(f'node {seed} choosed as seed')
        print(f'adjacent nodes to seed {seed} are {np.where(adj_matrix[seed]==1)[0]}')
        for p in np.where(adj_matrix[seed]==1)[0]:
            if marks[p] != 1:
                print(f'node {p} accepted as adjacent to seed {seed} and it is unmarked')
                marks[p] = 1
                Q_l.appendleft(p)
                print(f'node {p} marked and appended to Q_l')
                distances.appendleft(dists[seed, p])
            else:
                print(f'node {p} accepted as adjacent to seed {seed} but it is marked')
        print(f'adjcent nodes to seed {seed} all appended to Q_l. Now Q_l is : {Q_l}')
        while len(Q_l) != 0:
            p = Q_l.pop()
            print(f'node {p} poped from Q_l')
            temp_dist = distances.pop()
            if temp_dist < r:
                print(f'distance from node {p} to seed {seed} is: {temp_dist} below the threshold {r}') 
                count += 1
                S.add(p)
                print(f'node {p} added to set S. Now S is: {S}')
                print(f'adjacent nodes to {p} are {np.where(adj_matrix[p]==1)[0]}')
                for p_prime in np.where(adj_matrix[p]==1)[0]:
                    if marks[p_prime] != 1:
                        print(f'node {p_prime} accepted as adjacent to {p} and it is unmarked')
                        marks[p_prime] = 1
                        Q_l.appendleft(p_prime)
                        print(f'node {p_prime} marked and appended to Q_l')
                        distances.appendleft(dists[seed, p_prime])
                    else:
                        print(f'node {p_prime} accepted as adjacent to {p} but it is marked')
                print(f'adjcent nodes to {p} all appended to Q_l. Now Q_l is : {Q_l}')
                
                graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices}
                with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
                    pickle.dump(graph_dict, graph_file)
                sleep(0.5)

            else:
                print(f'distance from node {p} to seed {seed} is: {temp_dist} above the threshold {r}') 
                if count > delta:
                    print(f'number of nodes in S are: {count} above the threshold {delta}')
                    Vc = geo_center(S, V_set)
                    print(f'node {Vc} choosed as the geocenter node')
                    V_prime = adjacent_vertex(S, adj_matrix)
                    print(f'nodes {V_prime} are out of S')
                    ommited = list(S-{Vc})
                    print(f'nodes {ommited} ommited from V except geocenter {Vc}')
#                     mask = np.ones(indices.shape[0], dtype=bool)
#                     mask[ommited] = False
#                     indices = indices[mask]
                    indices = indices[~np.isin(indices, ommited)]
                    adj_matrix[:, ommited] = 0
                    adj_matrix[ommited] = 0
                    for v in V_prime:
                        Q_g.appendleft(v)
                        print(f'node {v} from V_prime appended to Q_g')
                        adj_matrix[v, Vc] = 1
                        adj_matrix[Vc, v] = 1
                    count = 0
                    S = set()
                    print('set S got empty')
                else:
                    print(f'number of nodes in S are: {count} below the threshold {delta}')

                graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices}
                with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
                    pickle.dump(graph_dict, graph_file)
                sleep(0.5)

                # eddited on the original algorithm
                all_marked = all_marked.union(set(np.where(marks==1)[0]))
                while len(Q_l) != 0:
                    p = Q_l.pop()
                    print(f'node {p} poped from Q_l and unmarked')
                    marks[p] = 0
                distances.clear()
                
            all_marked = all_marked.union(set(np.where(marks==1)[0]))
            print('*'*50)
        
        # eddited on the original algorithm
        count = 0
        S = set()
    
    all_marked = all_marked.union(set(np.where(marks==1)[0]))
    print(f'{"="*60} all marked are: {all_marked}')
    print(f'{"="*60} all nodes: {all_nodes}')
    print(f'{"="*60} random choices for s: {random_choices}')
    
    
# graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices}
# with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
#     pickle.dump(graph_dict, graph_file)

set S is: {290}
Q_g currently is: deque([290])
Q_l currently is: deque([290])
node 290 choosed as seed
adjacent nodes to seed 290 are [  5  23  80 108 139 150 206 228 231 239 273 280 302 317 331]
node 5 accepted as adjacent to seed 290 and it is unmarked
node 5 marked and appended to Q_l
node 23 accepted as adjacent to seed 290 and it is unmarked
node 23 marked and appended to Q_l
node 80 accepted as adjacent to seed 290 and it is unmarked
node 80 marked and appended to Q_l
node 108 accepted as adjacent to seed 290 and it is unmarked
node 108 marked and appended to Q_l
node 139 accepted as adjacent to seed 290 and it is unmarked
node 139 marked and appended to Q_l
node 150 accepted as adjacent to seed 290 and it is unmarked
node 150 marked and appended to Q_l
node 206 accepted as adjacent to seed 290 and it is unmarked
node 206 marked and appended to Q_l
node 228 accepted as adjacent to seed 290 and it is unmarked
node 228 marked and appended to Q_l
node 231 accepted as adjacent to see

**************************************************
node 150 poped from Q_l
distance from node 150 to seed 290 is: 0.0005756005559390387 below the threshold 0.001
node 150 added to set S. Now S is: {290, 5, 139, 108, 80, 150, 23}
adjacent nodes to 150 are [  5  23  80 108 228 239 273 280 290 317]
node 5 accepted as adjacent to 150 but it is marked
node 23 accepted as adjacent to 150 but it is marked
node 80 accepted as adjacent to 150 but it is marked
node 108 accepted as adjacent to 150 but it is marked
node 228 accepted as adjacent to 150 but it is marked
node 239 accepted as adjacent to 150 but it is marked
node 273 accepted as adjacent to 150 but it is marked
node 280 accepted as adjacent to 150 but it is marked
node 290 accepted as adjacent to 150 but it is marked
node 317 accepted as adjacent to 150 but it is marked
adjcent nodes to 150 all appended to Q_l. Now Q_l is : deque([253, 125, 336, 287, 257, 230, 331, 317, 302, 280, 273, 239, 231, 228, 206])
*****************************

**************************************************
node 317 poped from Q_l
distance from node 317 to seed 290 is: 0.00041188469259820367 below the threshold 0.001
node 317 added to set S. Now S is: {290, 228, 5, 231, 139, 108, 206, 239, 80, 273, 302, 150, 23, 280, 317}
adjacent nodes to 317 are [  5  23  80 108 150 228 239 273 280 290 331]
node 5 accepted as adjacent to 317 but it is marked
node 23 accepted as adjacent to 317 but it is marked
node 80 accepted as adjacent to 317 but it is marked
node 108 accepted as adjacent to 317 but it is marked
node 150 accepted as adjacent to 317 but it is marked
node 228 accepted as adjacent to 317 but it is marked
node 239 accepted as adjacent to 317 but it is marked
node 273 accepted as adjacent to 317 but it is marked
node 280 accepted as adjacent to 317 but it is marked
node 290 accepted as adjacent to 317 but it is marked
node 331 accepted as adjacent to 317 but it is marked
adjcent nodes to 317 all appended to Q_l. Now Q_l is : deque([356, 3

**************************************************
node 286 poped from Q_l
distance from node 286 to seed 257 is: 0.0008087205945208184 below the threshold 0.001
node 286 added to set S. Now S is: {271, 125, 247, 253, 286}
adjacent nodes to 286 are [ 43 104 125 188 193 218 230 247 253 257 271 277 287 334 336]
node 43 accepted as adjacent to 286 but it is marked
node 104 accepted as adjacent to 286 and it is unmarked
node 104 marked and appended to Q_l
node 125 accepted as adjacent to 286 but it is marked
node 188 accepted as adjacent to 286 but it is marked
node 193 accepted as adjacent to 286 but it is marked
node 218 accepted as adjacent to 286 but it is marked
node 230 accepted as adjacent to 286 but it is marked
node 247 accepted as adjacent to 286 but it is marked
node 253 accepted as adjacent to 286 but it is marked
node 257 accepted as adjacent to 286 but it is marked
node 271 accepted as adjacent to 286 but it is marked
node 277 accepted as adjacent to 286 but it is marked
node

**************************************************
node 338 poped from Q_l
distance from node 338 to seed 356 is: 0.00043202893421267043 below the threshold 0.001
node 338 added to set S. Now S is: {112, 338, 303}
adjacent nodes to 338 are [244 303 316 347 356]
node 244 accepted as adjacent to 338 but it is marked
node 303 accepted as adjacent to 338 but it is marked
node 316 accepted as adjacent to 338 but it is marked
node 347 accepted as adjacent to 338 but it is marked
node 356 accepted as adjacent to 338 but it is marked
adjcent nodes to 338 all appended to Q_l. Now Q_l is : deque([320, 316, 244, 120, 356, 347])
**************************************************
node 347 poped from Q_l
distance from node 347 to seed 356 is: 0.0002740456166380764 below the threshold 0.001
node 347 added to set S. Now S is: {112, 338, 347, 303}
adjacent nodes to 347 are [112 290 338 356]
node 112 accepted as adjacent to 347 but it is marked
node 290 accepted as adjacent to 347 but it is marked
node 

...

node 10 poped from Q_l and unmarked
node 18 poped from Q_l and unmarked
node 94 poped from Q_l and unmarked
node 109 poped from Q_l and unmarked
node 129 poped from Q_l and unmarked
node 137 poped from Q_l and unmarked
node 138 poped from Q_l and unmarked
node 154 poped from Q_l and unmarked
node 158 poped from Q_l and unmarked
node 168 poped from Q_l and unmarked
node 252 poped from Q_l and unmarked
node 323 poped from Q_l and unmarked
node 140 poped from Q_l and unmarked
node 187 poped from Q_l and unmarked
node 196 poped from Q_l and unmarked
node 318 poped from Q_l and unmarked
node 352 poped from Q_l and unmarked
node 353 poped from Q_l and unmarked
node 258 poped from Q_l and unmarked
node 289 poped from Q_l and unmarked
node 91 poped from Q_l and unmarked
node 14 poped from Q_l and unmarked
node 16 poped from Q_l and unmarked
node 17 poped from Q_l and unmarked
node 97 poped from Q_l and unmarked
node 264 poped from Q_l and unmarked
node 227 poped from Q_l and unmarked
node 249 

**************************************************
node 353 poped from Q_l
distance from node 353 to seed 258 is: 0.0004696221885750357 below the threshold 0.001
node 353 added to set S. Now S is: {224, 289, 353, 138, 50, 279, 252, 351}
adjacent nodes to 353 are [ 13  50  89 138 224 237 252 258 289]
node 13 accepted as adjacent to 353 but it is marked
node 50 accepted as adjacent to 353 but it is marked
node 89 accepted as adjacent to 353 but it is marked
node 138 accepted as adjacent to 353 but it is marked
node 224 accepted as adjacent to 353 but it is marked
node 237 accepted as adjacent to 353 but it is marked
node 252 accepted as adjacent to 353 but it is marked
node 258 accepted as adjacent to 353 but it is marked
node 289 accepted as adjacent to 353 but it is marked
adjcent nodes to 353 all appended to Q_l. Now Q_l is : deque([255, 194, 158, 109, 18, 348, 258, 143])
**************************************************
node 143 poped from Q_l
distance from node 143 to seed 258 is: 

node 14 poped from Q_l and unmarked
node 16 poped from Q_l and unmarked
node 17 poped from Q_l and unmarked
node 18 poped from Q_l and unmarked
node 91 poped from Q_l and unmarked
node 94 poped from Q_l and unmarked
node 97 poped from Q_l and unmarked
node 109 poped from Q_l and unmarked
node 129 poped from Q_l and unmarked
node 137 poped from Q_l and unmarked
node 140 poped from Q_l and unmarked
node 154 poped from Q_l and unmarked
node 158 poped from Q_l and unmarked
node 168 poped from Q_l and unmarked
node 187 poped from Q_l and unmarked
node 196 poped from Q_l and unmarked
node 227 poped from Q_l and unmarked
node 249 poped from Q_l and unmarked
node 258 poped from Q_l and unmarked
node 264 poped from Q_l and unmarked
node 318 poped from Q_l and unmarked
node 352 poped from Q_l and unmarked
**************************************************
Q_g currently is: deque([255, 158, 348, 89, 18, 109, 143, 237, 13, 194, 258, 318, 252, 187, 249, 237, 109, 168, 227, 353, 97, 289, 352, 94, 15

**************************************************
node 14 poped from Q_l
distance from node 14 to seed 129 is: 0.00047735626109208975 below the threshold 0.001
node 14 added to set S. Now S is: {1, 14}
adjacent nodes to 14 are [  1   4  10  12  16  17  18  19  20  26  33  46  63  87  89  94  97 109
 123 129 133 137 142 154 158 168 169 202 215 233 264]
node 1 accepted as adjacent to 14 but it is marked
node 4 accepted as adjacent to 14 but it is marked
node 10 accepted as adjacent to 14 but it is marked
node 12 accepted as adjacent to 14 but it is marked
node 16 accepted as adjacent to 14 but it is marked
node 17 accepted as adjacent to 14 but it is marked
node 18 accepted as adjacent to 14 but it is marked
node 19 accepted as adjacent to 14 but it is marked
node 20 accepted as adjacent to 14 but it is marked
node 26 accepted as adjacent to 14 but it is marked
node 33 accepted as adjacent to 14 but it is marked
node 46 accepted as adjacent to 14 but it is marked
node 63 accepted as adj

**************************************************
node 20 poped from Q_l
distance from node 20 to seed 129 is: 0.0007654129604369119 below the threshold 0.001
node 20 added to set S. Now S is: {1, 14, 16, 17, 18, 20}
adjacent nodes to 20 are [  1   4  10  12  14  16  17  18  19  26  46  63  87  94  97 102 117 123
 129 142 158 168 169 180 202 215 233 264]
node 1 accepted as adjacent to 20 but it is marked
node 4 accepted as adjacent to 20 but it is marked
node 10 accepted as adjacent to 20 but it is marked
node 12 accepted as adjacent to 20 but it is marked
node 14 accepted as adjacent to 20 but it is marked
node 16 accepted as adjacent to 20 but it is marked
node 17 accepted as adjacent to 20 but it is marked
node 18 accepted as adjacent to 20 but it is marked
node 19 accepted as adjacent to 20 but it is marked
node 26 accepted as adjacent to 20 but it is marked
node 46 accepted as adjacent to 20 but it is marked
node 63 accepted as adjacent to 20 but it is marked
node 87 accepted as 